# Data Cleaning Project: Cafe Sales

**Objective:** To perform a complete data cleaning process.

**Index:**
* Introduction: Import libraries and load the "dirty" .csv file;
* Step 1: Initial dataset diagnosis;
* Step 2: Adjusting Data Types;
* Step 3: Standardizing errors;
* Step 4: Correcting errors;
* Step 5: Checking for Duplicated Data;
* Step 6: Final Validation;
* Step 7: Save a new clean .csv file.

In [31]:
# Introduction

# Import Pandas and Numpy
import pandas as pd
import numpy as np

# Load the csv file
file_path = "../data/raw/dirty_cafe_sales.csv"
try:
    df = pd.read_csv(file_path)
    print("File loaded successfully!")
except FileNotFoundError:
    print("Error: File not found!")

# Display the first 5 rows
display(df.head())

File loaded successfully!


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


## Step 1: Initial Dataset Diagnosis

Before we change anything, let's first perform a complete "checkup" of our DataFrame to understand where the problems are!

In [32]:
# Technical summary: data types and non-null count
print("================== GENERAL INFORMATION ==================\n")
df.info()

# Count of missing (null) values in each column
print("\n========== COUNT OF MISSING (NULL) VALUES ==========")
with pd.option_context('display.max_rows', None):
    display(df.isnull().sum())

================== GENERAL INFORMATION ==================

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB

========== COUNT OF MISSING (NULL) VALUES ==========


Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

### Issues:

**Missing Values:** The presence of null values is noted in all columns (except for 'Transaction ID'), in addition to errors masked as 'UNKNOWN' and 'ERROR'.

**Column Types:** The 'Quantity', 'Price Per Unit', 'Total Spent', and 'Transaction Date' columns are of the object (String) type.

By converting them to the correct types (Float and DateTime, respectively), non-obvious errors like the ones mentioned above will automatically become null, which will help us!

## Step 2: Adjusting Data Types

As mentioned above, converting the columns to their respective types will be a powerful and beneficial move for when we actually clean the data.

Here, we will convert the 'Quantity', 'Price Per Unit', and 'Total Spent' columns to float, and the 'Transaction Date' column to DateTime.

Since we will actually be altering our DataFrame here, we will copy it to 'df_clean' so as not to modify our original data.

In [33]:
# Create a copy to work safely
df_clean = df.copy()

# Correcting numeric columns
numeric_columns = ['Price Per Unit', 'Quantity', 'Total Spent']

for column in numeric_columns:
    df_clean[column] = pd.to_numeric(df_clean[column], errors='coerce')

# Correcting Date column
df_clean['Transaction Date'] = pd.to_datetime(df_clean['Transaction Date'], errors='coerce')

print("Column types changed!\n")

Column types changed!



### Note:
Notice that we have now uncovered more errors in the columns that had the wrong data types. Take a look:

In [34]:
# Info before the transformation and the non-null count:
print("\n========== BEFORE ==========\n")
df.info()

# Info after the transformation and the non-null count:
print("\n========== AFTER ==========\n")
df_clean.info()


========== BEFORE ==========

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB

========== AFTER ==========

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    10000 non-null  object        
 1   Item              9667 non-null   object        
 2   Qu

## Step 3: Standardizing Errors

Standardizing errors is crucial to ensure the **consistency and accuracy** of your data, which leads to more reliable analyses. This **facilitates the automation** of cleaning processes and allows the team to focus more time on analysis rather than on manual correction.

In [35]:
values_to_treat_as_null = ['', ' ', 'UNKNOWN', 'ERROR']

# Standardizing all types of null values
df_clean.replace(values_to_treat_as_null, np.nan, inplace=True)

print("Missing values have been standardized!")

display(df_clean.head(7))

Missing values have been standardized!


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,NaN,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,NaN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5.0,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,NaN,3.0,3.0,9.0,NaN,Takeaway,2023-10-06


## Step 4: Correcting Errors

Before we handle the null values, we need to classify the columns into levels of relevance for our analysis.

**Essential Columns:** If a value is missing in any of these columns, the entire row becomes practically useless for most business analyses.

**Important Columns:** These columns are extremely important, but in some scenarios, their absence can be worked around (through filling/imputation) without completely invalidating the sales record.

**Contextual Columns:** These columns add valuable detail and context, but their absence does not invalidate the transaction record in any way.

**Derived Columns:** These are columns that are a function of others.

The relevance level of a column defines how "Null" values will be handled. For Essential columns: the entire row will be removed. For Important and Contextual columns: they will be filled with the Mode. For Derived columns: a validation will be performed.

In [36]:
# Classifying the Columns
essential_columns = ['Transaction Date', 'Price Per Unit', 'Quantity', 'Item']
important_columns = ['Transaction ID'] # Since there are no nulls in this column, I won't use it
contextual_columns = ['Location', 'Payment Method']

# ========== Handling Nulls in Essential Columns ==========
# Removing all rows that have null values in the essential columns
df_clean.dropna(subset=essential_columns, inplace=True)

# ========== Handling Nulls in Contextual Columns ==========
# Loop to handle each column in the list
for column in contextual_columns:
    # Calculate the mode (the most frequent value)
    # We use .mode()[0] because .mode() returns a Series (in case of a tie for the mode),
    # and we want to get only the first value.
    mode = df_clean[column].mode()[0]

    # Fill the null values (NaN) with the mode
    df_clean[column] = df_clean[column].fillna(mode)

    print(f"Nulls in column '{column}' filled with the mode: '{mode}'")

# ========== Handling the 'Total Spent' column ==========
# Prerequisite: The 'Quantity' and 'Price Per Unit' columns must already be clean
# (types corrected, nulls handled, etc.)

# 1. Calculate a new 'calculated_total' column for verification
#    This allows us to compare with the original 'Total Spent' column.
df_clean['calculated_total'] = df_clean['Quantity'] * df_clean['Price Per Unit']

# 2. Replace the original column with the calculated one
df_clean['Total Spent'] = df_clean['calculated_total']

# 3. Final Cleanup: Remove the temporary columns we created
df_clean.drop(columns=['calculated_total'], inplace=True)

print("\n'Total Spent' column validated, rebuilt, and consistent!")

print(f"Checking if nulls still exist in 'Total Spent' (there shouldn't be any): {df_clean['Total Spent'].isnull().sum()}")

Nulls in column 'Location' filled with the mode: 'In-store'
Nulls in column 'Payment Method' filled with the mode: 'Digital Wallet'

'Total Spent' column validated, rebuilt, and consistent!
Checking if nulls still exist in 'Total Spent' (there shouldn't be any): 0


## Step 5: Checking for Duplicate Data

Now that there are no longer any null values or inconsistencies, we can check for duplicate rows. If there are any, we will need to remove them.

In [37]:
# Counting duplicate rows
print("\n============== COUNTING DUPLICATE ROWS ==============")
num_duplicates = df.duplicated().sum()
print(f"The dataset has {num_duplicates} duplicate rows.")


============== COUNTING DUPLICATE ROWS ==============
The dataset has 0 duplicate rows.


## Step 6: Final Validation

Since there are no duplicate rows, we can say that our data is clean and ready to be used for analysis! Take a look at what our clean DataFrame looks like now:

In [38]:
# Technical summary: data types and non-null count
print("================== GENERAL INFORMATION ==================\n")
df_clean.info()

# Count of missing (null) values in each column
print("\n========== COUNT OF MISSING (NULL) VALUES ==========")
with pd.option_context('display.max_rows', None):
    display(df_clean.isnull().sum())

================== GENERAL INFORMATION ==================

<class 'pandas.core.frame.DataFrame'>
Index: 7773 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    7773 non-null   object        
 1   Item              7773 non-null   object        
 2   Quantity          7773 non-null   float64       
 3   Price Per Unit    7773 non-null   float64       
 4   Total Spent       7773 non-null   float64       
 5   Payment Method    7773 non-null   object        
 6   Location          7773 non-null   object        
 7   Transaction Date  7773 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 546.5+ KB

========== COUNT OF MISSING (NULL) VALUES ==========


Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

## Step 7: Save a New, Clean .csv File

With the entire data cleaning process completed, we just need to save our new .csv file to be used!

In [39]:
# Save the clean CSV file
final_file_name = '../data/processed/cleaned_cafe_sales.csv'
df_clean.to_csv(final_file_name, index=False, encoding='utf-8')
print(f"\nFile '{final_file_name}' saved successfully!")


File '../data/processed/cleaned_cafe_sales.csv' saved successfully!
